#Links-
- https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python

# Initial setup

In [ ]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""


In [ ]:
%pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for num2words from https://files.pythonhosted.org/packages/8f/f0/ca1228af2bcbce2fdf2b23d58643c84253b88a3c1cd9dba391ca683c4b21/num2words-0.5.13-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/143.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 133.1/143.3 kB 16.4 MB/s eta 0:00:01
   ━━━━━━━━━━━

In [ ]:
dbutils.library.restartPython()

In [ ]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

# Getting the data

In [ ]:

df=pd.read_csv('/Volumes/genai_exploration/default/file/bill_sum_data.csv') # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [ ]:
df_bills = df[['text', 'summary', 'title']]
df_bills

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...
7,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993


In [ ]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

# Sample embedding using tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

20

In [ ]:
df_bills

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,2026
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,880
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,2815
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,2479
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947


In [ ]:
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'SECTION',
 b' ',
 b'1',
 b'.',
 b' SHORT',
 b' TITLE',
 b'.',
 b' This',
 b' Act',
 b' may',
 b' be',
 b' cited',
 b' as',
 b' the',
 b' ``',
 b'National',
 b' Science',
 b' Education',
 b' Tax',
 b' In',
 b'cent',
 b'ive',
 b' for',
 b' Businesses',
 b' Act',
 b' of',
 b' ',
 b'200',
 b'7',
 b"''.",
 b' SEC',
 b'.',
 b' ',
 b'2',
 b'.',
 b' C',
 b'RED',
 b'ITS',
 b' FOR',
 b' CERT',
 b'AIN',
 b' CONTRIBUT',
 b'IONS',
 b' BEN',
 b'EF',
 b'IT',
 b'ING',
 b' SC',
 b'IENCE',
 b',',
 b' TECHNO',
 b'LOGY',
 b',',
 b' ENGINE',
 b'ERING',
 b',',
 b' AND',
 b' MAT',
 b'HE',
 b'MAT',
 b'ICS',
 b' EDUC',
 b'ATION',
 b' AT',
 b' THE',
 b' ELEMENT',
 b'ARY',
 b' AND',
 b' SECOND',
 b'ARY',
 b' SCHOOL',
 b' LEVEL',
 b'.',
 b' (',
 b'a',
 b')',
 b' In',
 b' General',
 b'.--',
 b'Sub',
 b'part',
 b' D',
 b' of',
 b' part',
 b' IV',
 b' of',
 b' sub',
 b'chapter',
 b' A',
 b' of',
 b' chapter',
 b' ',
 b'1',
 b' of',
 b' the',
 b' Internal',
 b' Revenue',
 b' Code',
 b' of',
 b' ',
 b'198',
 b'6',


# Azure open ai embeddings generation

In [ ]:
# Using azure openai to generate embeddings
client = AzureOpenAI(
  # api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_key="",
  api_version = "2024-02-01",
  # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
  azure_endpoint="h"
)

def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
print(df_bills)

                                                 text  ...                                             ada_v2
0   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.022682489827275276, -0.008019787259399891,...
1   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.041704513132572174, -0.009070469066500664,...
2   SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...  ...  [-0.04189346358180046, -0.0024857877288013697,...
3   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.013219092972576618, -0.007222938816994429,...
4   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.03924504294991493, -0.0255487859249115, -0...
5   SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...  ...  [-0.01617865450680256, -0.01530303992331028, -...
6   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.029890764504671097, -0.02193722128868103, ...
7   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.020711546763777733, 0.0001716967235552147,...
8   SECTIO

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

text summary title n_tokens ada_v2 similarities SECTION 1. SHORT TITLE. This Act may be cited as the ``Taxpayer's Right To View Act of 1993''. SEC. 2. FINDINGS. The Congress finds the following: (1) The Federal Communications Commission predicts a diminished role for broadcast television and a more prominent role for cable television including pay-per-view. (2) Roughly 18,800,000 American homes are equipped to receive pay-per-view and this number is expected to increase to 35,900,000 by 1996. Overall pay-per-view revenue is expected to reach $1,100,000,000 by 1996. (3) There is a growing trend toward making events available exclusively on pay-per-view. (4) As this trend develops, whether the consumer has access to these events will be determined by the ability of the consumer to pay. (5) Professional sports leagues are beginning to see pay- per-view as a new revenue source to keep pace with escalating player salaries. (6) As a result, some media analysts predict that several of broadcast television's premier sports attractions eventually will migrate to pay-per-view. (7) Limited access to viewing such events as the ``Super Bowl'' or ``World Series'' would deprive citizens of the ability to enjoy these events which are inherent in the American tradition. (8) The majority of facilities in which such events are held are funded through taxpayer money. (9) It is unfair that taxpayers, who subsidize the construction and maintenance of many of these facilities, should have to pay an additional pay-per-view charge for viewing these events. (10) Nonprofit and public organizations including public and private educational institutions and their athletic organizations are exempt from the corporate income tax. (11) Furthermore, corporations can deduct their donations to college sporting events as charitable contributions. (12) Tax exempt status is granted to these nonprofit and public organizations by the Federal Government and it is unfair to allow those who use this status to engage in pay-per-view telecasting--thus forcing taxpayers to pay again. (13) It is unfair that taxpayers, who subsidize the construction and maintenance of many of these facilities, should have to pay an additional pay-per-view charge for viewing these events. (14) Therefore, Congress should ensure that all taxpaying citizens have free access to events that are sponsored by organizations with nonprofit tax exempt status and those events held in taxpayer subsidized facilities. SEC. 3. PAY-PER-VIEW CHARGES PROHIBITED. Section 623 of the Communications Act of 1934 (47 U.S.C. 543) is amended by adding at the end thereof the following new subsection: ``(i)(1) Notwithstanding any other provision of this title, a cable operator may not assess or collect any separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. ``(2) The Commission and local franchising authorities are authorized to make determinations concerning the applicability of the prohibition contained in paragraph (1). In making such determinations-- ``(A) a facility shall be considered to have been constructed, maintained, or renovated with tax revenues if-- ``(i) tax exempt financing was used to construct, maintain, or renovate the facility, ``(ii) the facility was constructed on land donated by a government, or leased by a government at below market rates, or ``(iii) public infrastructure or public service for the facility are provided by the government at below market rates, with the exception of police, fire, and rescue services; ``(B) an event is performed by a nonprofit or public organization that receives tax subsidies if the event is sponsored by, or includes the participation of a team that is a part of, an organization-- ``(i) that is exempt from Federal income taxes under section 501 of the Internal Rev

In [ ]:
res["summary"][9]

"Taxpayer's Right to View Act of 1993 - Amends the Communications Act of 1934 to prohibit a cable operator from assessing separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. Authorizes the Federal Communications Commission and local franchising authorities to make determinations concerning the applicability of such prohibition. Sets forth conditions under which a facility is considered to have been constructed, maintained, or renovated with tax revenues. Considers events performed by nonprofit or public organizations that receive tax subsidies to be subject to this Act if the event is sponsored by, or includes the participation of a team that is part of, a tax exempt organization."

# Storing embeddings in databricks


In [ ]:
print(df_bills.columns)

Index(['text', 'summary', 'title', 'n_tokens', 'ada_v2', 'similarities'], dtype='object')


In [ ]:
new_df_bills = df_bills[['text','title', 'summary','ada_v2']]
print(new_df_bills)

                                                 text  ...                                             ada_v2
0   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.022682489827275276, -0.008019787259399891,...
1   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.041704513132572174, -0.009070469066500664,...
2   SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...  ...  [-0.04189346358180046, -0.0024857877288013697,...
3   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.013219092972576618, -0.007222938816994429,...
4   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.03924504294991493, -0.0255487859249115, -0...
5   SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...  ...  [-0.01617865450680256, -0.01530303992331028, -...
6   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.029890764504671097, -0.02193722128868103, ...
7   SECTION 1. SHORT TITLE. This Act may be cited ...  ...  [-0.020711546763777733, 0.0001716967235552147,...
8   SECTIO

In [ ]:
import pandas as pd
import pyspark.pandas as ps

#onvert to spark
spark_df_bills = ps.from_pandas(new_df_bills).to_spark()

In [ ]:
# Define the path to store the Delta table
delta_table_path = "/mnt/delta/bills_mbeddings_table"

# Write the Spark DataFrame to a Delta table
spark_df_bills.write.format("delta").mode("overwrite").save(delta_table_path)


In [ ]:
# Load the Delta table from the file path
delta_df = spark.read.format("delta").load(delta_table_path)

# Show the data
delta_df.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|               title|             summary|              ada_v2|
+--------------------+--------------------+--------------------+--------------------+
|SECTION 1. SHORT ...|To amend the Inte...|National Science ...|[-0.0226824898272...|
|SECTION 1. SHORT ...|To amend the Inte...|Small Business Ex...|[-0.0417045131325...|
|SECTION 1. RELEAS...|A bill to require...|Requires the Dire...|[-0.0418934635818...|
|SECTION 1. SHORT ...|A bill to improve...|National Cancer A...|[-0.0132190929725...|
|SECTION 1. SHORT ...|A bill to amend t...|Military Call-up ...|[-0.0392450429499...|
|SECTION 1. RELIQU...|To provide for re...|Requires the Cust...|[-0.0161786545068...|
|SECTION 1. SHORT ...|A bill to require...|Service Dogs for ...|[-0.0298907645046...|
|SECTION 1. SHORT ...|A bill to provide...|Race to the Top A...|[-0.0207115467637...|
|SECTION 1. SHORT ...|Troop Talent Act ...|Troop Talen

In [ ]:
# save as delta table
spark_df_bills.write.format("delta").mode("overwrite").saveAsTable("genai_exploration.default.bills_embeddings_table")

In [ ]:
# check if exists
spark.sql("SHOW TABLES").show()


+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|bills_embeddings_...|      false|
+--------+--------------------+-----------+



In [ ]:
# Use SQL to query the table
spark.sql("SELECT * FROM genai_exploration.default.bills_embeddings_table").show()

# List tables in the database to confirm
spark.sql("SHOW TABLES IN genai_exploration.default").show()

+--------------------+--------------------+--------------------+--------------------+
|                text|               title|             summary|              ada_v2|
+--------------------+--------------------+--------------------+--------------------+
|SECTION 1. SHORT ...|To amend the Inte...|National Science ...|[-0.0226824898272...|
|SECTION 1. SHORT ...|To amend the Inte...|Small Business Ex...|[-0.0417045131325...|
|SECTION 1. RELEAS...|A bill to require...|Requires the Dire...|[-0.0418934635818...|
|SECTION 1. SHORT ...|A bill to improve...|National Cancer A...|[-0.0132190929725...|
|SECTION 1. SHORT ...|A bill to amend t...|Military Call-up ...|[-0.0392450429499...|
|SECTION 1. RELIQU...|To provide for re...|Requires the Cust...|[-0.0161786545068...|
|SECTION 1. SHORT ...|A bill to require...|Service Dogs for ...|[-0.0298907645046...|
|SECTION 1. SHORT ...|A bill to provide...|Race to the Top A...|[-0.0207115467637...|
|SECTION 1. SHORT ...|Troop Talent Act ...|Troop Talen

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import FloatType, ArrayType
from pyspark.sql.functions import broadcast

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Load Delta table
df_bills_delta = spark.read.format("delta").table("genai_exploration.default.bills_embeddings_table")


# Define cosine similarity function
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# Define spark UDFs
cosine_similarity_udf = udf(cosine_similarity, FloatType())

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# Get embedding for user query
user_query = "Can I get information on cable company tax revenue?"
query_embedding = get_embedding(user_query)
query_embedding_list = [float(val) for val in query_embedding]

# Broadcast the query embedding
broadcast_embedding = spark.sparkContext.broadcast(query_embedding_list)

# Define a UDF to use the broadcasted embedding
def cosine_similarity_udf(ada_v2):
    embedding = broadcast_embedding.value
    return cosine_similarity(ada_v2, embedding)

cosine_similarity_spark_udf = udf(cosine_similarity_udf, FloatType())

# Add a column for similarities
df_bills_delta = df_bills_delta.withColumn("similarities", cosine_similarity_spark_udf(col("ada_v2")))

# Sort and select top_n
top_n = 4
result_df = df_bills_delta.orderBy(col("similarities").desc()).limit(top_n)

# Display results
result_df.show()


+--------------------+--------------------+--------------------+--------------------+------------+
|                text|               title|             summary|              ada_v2|similarities|
+--------------------+--------------------+--------------------+--------------------+------------+
|SECTION 1. SHORT ...|Taxpayer's Right ...|Taxpayer's Right ...|[-0.0185791235417...|   0.8191763|
|SECTION 1. SHORT ...|To amend the Inte...|Small Business Ex...|[-0.0417045131325...|   0.7341673|
|SECTION 1. SHORT ...|A bill to amend t...|Wall Street Compe...|[-0.0474344156682...|   0.7337838|
|SECTION 1. SHORT ...|To amend title XI...|Strengthening the...|[-0.0178938712924...|  0.73289365|
+--------------------+--------------------+--------------------+--------------------+------------+



# Storing embeddings in pinecone

In [ ]:
!pip install pinecone-client

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/55/d0/c64336b8f76e63296d04b885c545c0872ff070e6b2bc725dd0ff3ae681dc/pinecone_client-5.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pinecone-plugin-inference<2.0.0,>=1.0.3 from https://files.pythonhosted.org/packages/0d/b7/0d57cad06545ac8fbb7a362dddaff01b0ecfe6e47c135345e94b3d8ab2ca/pinecone_plugin_inference-1.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for pinecone-plugin-interface<0.0.8,>=0.0.7 from https://files.pythonhosted.org/packages/3b/1d/a21fdfcd6d022cb64cef5c2a29ee6691c6c103c4566b41646b080b7536a5/pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/244.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
# dbutils.library.restartPython()

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc_client = Pinecone(api_key="")

In [ ]:
index_name = 'bills-embedding-index'  

pc_client.create_index(
  name=index_name,
  dimension=len(new_df_bills['ada_v2'][0]),
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=

In [ ]:
# Convert new_df_bills to a list of dictionaries for Pinecone
items = [
    {
        'id': str(i),
        'values': new_df_bills.loc[i, 'ada_v2'],
        'metadata': {
            'text': new_df_bills.loc[i, 'text'],
            'title': new_df_bills.loc[i, 'title'],
            'summary': new_df_bills.loc[i, 'summary']
        }
    }
    for i in range(len(new_df_bills))
]
print(items)

---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=

In [ ]:
# Get the index
index = pc_client.Index(index_name)

# Upsert data into Pinecone
index.upsert(vectors=items)


---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=

In [ ]:
# Example query
query_vector = new_df_bills['ada_v2'][0]  # Use an example embedding vector
results = index.query(vector=query_vector, top_k=5)

# Print results
print(results)

# results = index.fetch(["7"])
# print(results)

---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=

In [ ]:
index.fetch(["7", "id-2"])

---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=

In [ ]:
# Print detailed results
print("Query Results:")
for match in results['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match.get('metadata', 'No metadata')}")
    print("-----")

---------------------------------------------------------------------------
PineconeApiException                      Traceback (most recent call last)
File <command-145194252121746>, line 3
      1 index_name = 'bills-embedding-index'  
----> 3 pc_client.create_index(
      4   name=index_name,
      5   dimension=len(new_df_bills['ada_v2'][0]),
      6   metric="cosine",
      7   spec=ServerlessSpec(
      8     cloud="aws",
      9     region="us-east-1"
     10   )
     11 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-399af6c0-798e-47e5-bfd3-09c12469efa2/lib/python3.11/site-packages/pinecone/control/pinecone.py:384, in Pinecone.create_index(self, name, dimension, spec, metric, timeout, deletion_protection)
    381 else:
    382     raise TypeError("spec must be of type dict, ServerlessSpec, or PodSpec")
--> 384 api_instance.create_index(
    385     create_index_request=CreateIndexRequest(
    386         name=name,
    387         dimension=dimension,
    388         metric=